In [1]:
import torch
from torch import tensor


In [2]:
from fastai.vision.all import *

pickle_path = URLs.path('mnist_png')/'mnist_png.pkl'
path = untar_data(URLs.MNIST)/'training'

if not pickle_path.exists():
    pickle_path.parent.mkdir(parents=True, exist_ok=True)
    ds = DataBlock(
        blocks = (ImageBlock(PILImageBW), CategoryBlock),
        get_items = get_image_files,
        get_y = parent_label,
        splitter = RandomSplitter(1/6, seed=0)
    ).datasets(path)

    xs, ys = zip(*ds.train, *ds.valid)
    xs = np.stack(L(map(lambda x: np.array(x, dtype=np.float32).reshape(-1), xs))) / 255.
    ys = np.array(ys, dtype=np.int64)

    x_train, x_valid = xs[:len(ds.train)], xs[len(ds.train):]
    y_train, y_valid = ys[:len(ds.train)], ys[len(ds.train):]

    save_pickle(pickle_path, [x_train, y_train, x_valid, y_valid])

    del ds, xs, ys, x_train, y_train, x_valid, y_valid

x_train, y_train, x_valid, y_valid = map(tensor, load_pickle(pickle_path))

torch.manual_seed(1)
weights = torch.randn(784, 10)
bias = torch.zeros(10)

m1 = x_valid[:5]
m2 = weights
ar, ac = m1.shape 
br, bc = m2.shape

t1 = torch.zeros(ar, bc)

for i in range(ar):         # 5
    for j in range(bc):     # 10
        for k in range(ac): # 784
            t1[i, j] += m1[i, k] * m2[k, j]


Einstein summation.

The dot product looks like `ik,kj->ij`. The arrays are multiplied along the 2nd dim of the 1st tensor and the 1st dim of the 2nd tensor, creating a 3-dim array that can be represented as `ijk`. Then the output array is summed along the 3rd dimension.

Specified in the format e.g. `ikl,lkj->ik`. Here, we take the 2nd and 3rd dims of the 1st array and the 2nd and 1st dims of the first array and form a product that can be represented as `ijkl`; we then sum along the `j` and the `l` dimension.

```
p = zeros(sza0, szb2, sza1, sza3)
for i in sza0:
    for all j in szb2:
        for all k in sza1:
            for all l in sza3:
                p[i, j, k, l] += a[i, k, l] * b[l, k, j]
return p.sum(dims=(1, 3))
```
there are more subtleties and aspects to einsum that are not mentioned here; see docs.

In [3]:
mr = torch.einsum('ik,kj->ikj', m1, m2)
mr.shape


torch.Size([5, 784, 10])

In [4]:
def matmul(a, b):
    return torch.einsum('ik,kj->ij', a, b)


In [5]:
%timeit -n 5 _ = matmul(x_train, weights)


28.2 ms ± 1.4 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)
